<div class="usecase-title">{Bins for events Argyle Squrae}</div>

<div class="usecase-authors"><b>Authored by: </b> {Alison Collins}</div>

<div class="usecase-duration"><b>Duration:</b> {90} mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>{Intermediate}</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>{Python}</div>
</div>

<div class="usecase-section-header">Scenario</div>

{If you are palnning an event at Argyle Square you will need to know if you need to hire additional bins. This use case seeks to determine th ebin capacity during events and make recommendations on the need for more bins based on event attendee numbers.}

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- {list the skills demonstrated in your use case}

<div class="usecase-section-header">{Heading for introduction or background relating to problem}</div>

{Write your introduction here. Keep it concise. We're not after "War and Peace" but enough background information to inform the reader on the rationale for solving this problem or background non-technical information that helps explain the approach. You may also wish to give information on the datasets, particularly how to source those not being imported from the client's open data portal.}



In [2]:
!jupyter nbconvert --to html usecase_TEMPLATE_COPY.ipynb

[NbConvertApp] Converting notebook usecase_TEMPLATE_COPY.ipynb to html
[NbConvertApp] Writing 588410 bytes to usecase_TEMPLATE_COPY.html
